# Modelagem à partir dos dados

Neste projeto um grafo será modelado para que a análise seja realizada através de algoritmos aplicados à essa estrutura. Portanto primeiramente este grafo será modelado com os dados exportados pelo [DataCleaning.ipynb](#https://github.com/jAzz-hub/DesmatamentoNoAmazonas/blob/main/Notebooks/DataCleaning.ipynb).

In [50]:
#Importando modulos:
from modules import *

DATASETS = []

#Recriando vetor com dados tratados
for i in range(0,19):
    if i < 10: DATASETS.append(pd.read_csv(f'../Data/CleanData2000_2018/Cleaned200{i}.csv', sep=',', encoding='utf-8'))
    elif i>=10: DATASETS.append(pd.read_csv(f'../Data/CleanData2000_2018/Cleaned20{i}.csv', sep=',', encoding='utf-8'))
     

DATASETS[0].head(3)



,Lat,Long,Municipio,AreaKm2,Desmatado2000,Floresta2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000,Fronteiras,MaiorQueAMédia
0,-2.83490,-57.83095,Urucurituba,2915,82.3,142.7,0.0,1689.6,1000.4,"['Boa Vista do Ramos', 'Barreirinha', 'Itapira...",False
1,-1.24938,-58.78714,Urucara,28012,299.2,26642.5,0.9,423.0,646.4,"['Nhamunda', 'Urucurituba', 'Itapiranga', 'Sao...",False
2,-3.20648,-65.43058,Uarini,10448,194.2,9721.7,0.0,100.3,431.8,"['Maraa', 'Fonte Boa', 'Jurua', 'Alvaraes', 'T...",False


### Função que cria arestas dos nós de cidades vizinhas:

A função recebe um `dataFrame` e estabelece as arestas(adjascência) entre nós(municípios) com base nas nas listas da coluna `Fronteira`.


In [ ]:

def LinkGenerator(dataFrame):
    
    Mapa = nx.Graph()
    
    for line in range(0, len(dataFrame['Municipio'])):
        neighbors = [ast.literal_eval(neighbor) for neighbor in dataFrame['Fronteiras']]
        
        
        for cityNeighbor in neighbors[line]:
            
            if not Mapa.has_edge(dataFrame['Municipio'][line], cityNeighbor) and  not Mapa.has_edge(cityNeighbor, dataFrame['Municipio'][line]):        
                Mapa.add_edge(dataFrame['Municipio'][line], cityNeighbor)#Fronteiras 
            
            else:
                pass
            
    return Mapa


In [ ]:

#Criando Grafos para cada ano:
Graphs = []
for i in range(0, len(DATASETS)):
    Graphs.append(LinkGenerator(DATASETS[i]))

#Mostrando quantidade de nós e arestas em um grafo do ano 2000:
print(len(Graphs[0].nodes()))
print(len(Graphs[0].edges()))



In [ ]:


def AverageHigh(city, dataFrame):
    for index in range(0, len(dataFrame['Municipio'])):
        return dataFrame['Municipio'][index] == city and  dataFrame['MaiorQueAMédia'][index]
    
    return False   

#Ideia da função que cria grafo com arestas acima da média:    
# def LinkGreaterThanAverageGenerator(cidade, fronteiras, MapaDeRisco):
    #pega a cidade:
    #verifica se o Desmatamento está acima da média:
    #se estiver gera arestas entre ela e seuas fronteiras com desmatamento acima da média: